# Introduction

Did I generate that merged archive correctly?

In [1]:
import csv
from collections import Counter
import scanpy
import pysam
from pathlib import Path
import pandas
import requests
import numpy
import os
from io import BytesIO
import re
import shutil
import scipy
import gzip
import sys
from matplotlib import pyplot
import matplotlib
import upsetplot
import warnings
from urllib.parse import urljoin
import yaml

In [2]:
MEX = str(Path("~/proj/mex_gene_archive").expanduser())
if MEX not in sys.path:
    sys.path.append(MEX)
from mex_gene_archive.filter import write_filtered_mtx
from mex_gene_archive.splitseq import (
    archive_split_seq_solo,
    load_merged_parse_matrix,
    make_list_of_archive_files,
    SimpleMMWriter
)
from mex_gene_archive.reader import read_mex_archive, read_mex_archive_as_anndata


In [3]:
star_analysis_dir = Path("adrenal/mouse_adr_2k_a18m_m1_pipeline")

In [4]:
em_filtered = star_analysis_dir / "GeneFull_Ex50pAS_EM_filtered.tar.gz"

In [5]:
components = read_mex_archive(em_filtered)

In [6]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [7]:
components.keys()

dict_keys(['metadata', 'barcodes', 'features', 'matrix'])

In [8]:
components["barcodes"].shape

(448,)

In [9]:
components["features"].shape

(81881, 2)

In [10]:
components["matrix"].shape

(81881, 448)

In [11]:
assert components["matrix"].shape[1] != components["barcodes"].shape[0], "hey we fixed it."


AssertionError: hey we fixed it.

In [ ]:
solo_root = star_analysis_dir / "Solo.out"
quantification="GeneFull_Ex50pAS"
multiread="EM"
matrix="filtered"

In [ ]:
X, cells = load_merged_parse_matrix(solo_root, quantification=quantification, multiread=multiread, matrix=matrix) 

In [ ]:
X.shape, len(cells)

In [ ]:
archive_files = {}
for pathname in make_list_of_archive_files(solo_root, quantification, multiread, matrix):
    name = pathname.name
    if name == "barcodes.tsv":
        value = BytesIO(os.linesep.encode("ascii").join([x.encode("ascii") for x in cells]))
    elif name == "features.tsv":
        value = pathname
    else:
        value = BytesIO()
        SimpleMMWriter().write(value, X, comment='', field=None, precision=None, symmetry=None)
        value.seek(0)
    archive_files[pathname] = value



In [ ]:
archive_files

In [ ]:
matrix_key = Path('adrenal/mouse_adr_2k_a18m_m1_pipeline/Solo.out/GeneFull_Ex50pAS/filtered/UniqueAndMult-EM.mtx')
print(len(archive_files[matrix_key].getvalue()))
print(archive_files[matrix_key].getvalue().decode("ascii")[-1000:])

In [ ]:
with open(star_analysis_dir/"config.yaml", "rt") as instream:
    config = yaml.load(instream, Loader=yaml.SafeLoader)

In [ ]:
archive_split_seq_solo(star_analysis_dir/"Solo.out", config, "GeneFull_Ex50pAS", "EM", "filtered", destination=star_analysis_dir / "GeneFull_Ex50pAS_Unique_filtered_notebook.tar.gz")

In [ ]:
b = BytesIO()

In [ ]:
for i in range(5):
    b.write(b"Hello\n")

In [ ]:
b.tell()

In [ ]:
b.seek(0)

In [ ]:
with open("/dev/shm/bytes.test.txt", "wb") as outstream:
    shutil.copyfileobj(b, outstream)